# Import Library

In [1]:
!pip install ultralytics opencv-python tqdm huggingface_hub

Defaulting to user installation because normal site-packages is not writeable
  Using cached numpy-2.2.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2


In [2]:
! pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121 # cuda 12.1 기준

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://download.pytorch.org/whl/cu121


In [3]:
! pip install numpy==1.25.2 --force-reinstall

Defaulting to user installation because normal site-packages is not writeable
  Using cached numpy-1.25.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.2 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.25.2 which is incompatible.


In [4]:
import os
import json
import cv2
from glob import glob
from tqdm import tqdm
from ultralytics import YOLO
import numpy as np

from datetime import datetime

# 2. Json -> YOLO txt 변환

In [5]:
def convert_to_yolo(json_file_path, output_dir):
    """
    JSON 파일을 읽어 YOLO 포맷의 txt 파일로 변환합니다.
    """
    try:
        with open(json_file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        # 이미지 크기 정보 추출
        resolution = data.get("meta", {}).get("Resolution", "0x0").split('x')
        if len(resolution) != 2:
            # print(f"Skipping {json_file_path}: Invalid resolution format.")
            return
            
        img_width, img_height = int(resolution[0]), int(resolution[1])
        if img_width == 0 or img_height == 0:
            # print(f"Skipping {json_file_path}: Image width or height is zero.")
            return

        # 어노테이션 정보 추출
        annotations = data.get("annotations", {}).get("Bbox Annotation", {})
        boxes = annotations.get("Box", [])
        
        if not boxes:
            # Bounding Box가 없는 경우, 빈 txt 파일 생성
            base_filename = os.path.basename(json_file_path)
            txt_filename = os.path.splitext(base_filename)[0] + ".txt"
            open(os.path.join(output_dir, txt_filename), 'w').close()
            return

        yolo_data = []
        for box in boxes:
            # YOLO 포맷으로 변환
            # 현재는 단일 클래스(0)로 처리
            class_id = 0  
            x, y, w, h = box['x'], box['y'], box['w'], box['h']
            
            x_center = (x + w / 2) / img_width
            y_center = (y + h / 2) / img_height
            width_norm = w / img_width
            height_norm = h / img_height
            
            yolo_data.append(f"{class_id} {x_center:.6f} {y_center:.6f} {width_norm:.6f} {height_norm:.6f}")

        # YOLO 포맷으로 파일 저장
        base_filename = os.path.basename(json_file_path)
        txt_filename = os.path.splitext(base_filename)[0] + ".txt"
        
        with open(os.path.join(output_dir, txt_filename), 'w') as f:
            f.write("\n".join(yolo_data))

    except Exception as e:
        print(f"Error processing {json_file_path}: {e}")

def process_dataset(dataset_path):
    """
    'train'과 'val' 폴더에 대해 변환 작업을 수행합니다.
    """
    for split in ["train", "val"]:
        json_dir = os.path.join(dataset_path, split, "annotations")
        output_dir = os.path.join(dataset_path, split, "labels")

        if not os.path.exists(json_dir):
            print(f"Directory not found: {json_dir}")
            continue

        os.makedirs(output_dir, exist_ok=True)
        
        json_files = glob(os.path.join(json_dir, "*.json"))
        print(f"Found {len(json_files)} json files in {json_dir}")

        for json_file in tqdm(json_files, desc=f"Converting {split} JSONs to YOLO format"):
            convert_to_yolo(json_file, output_dir)
        
        print(f"Finished processing for {split} set. Labels are in {output_dir}")

In [6]:
# 한번만 실행
root_path = "C:/Users/chobh/Desktop/빅프로젝트"
dataset_base_path = os.path.join(root_path, "custom_dataset")
process_dataset(dataset_base_path)

Directory not found: C:/Users/chobh/Desktop/빅프로젝트/custom_dataset/train/annotations
Directory not found: C:/Users/chobh/Desktop/빅프로젝트/custom_dataset/val/annotations


# 3. YOLO Fine tuning

In [7]:
model = YOLO('yolov8n.pt')  # nano, 또는 yolov8s.pt 등 다른 가중치 사용 가능

In [16]:
results = model.train(
    data=os.path.join('./data.yaml'),
    epochs=1,       # 필요에 따라 조정
    imgsz=640,
    batch=16,        # 메모리에 따라 조정
    device="cpu"        # CPU로 설정// # GPU 사용
)


New https://pypi.org/project/ultralytics/8.3.169 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.168 🚀 Python-3.10.12 torch-2.5.1+cu121 CPU (AMD EPYC 7B13)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=./data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=1, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train15, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overla

RuntimeError: Dataset 'data.yaml' error ❌ Dataset 'data.yaml' images not found, missing path '/workspace/Illegal-Parking-Detection/AI/Detection/custom_dataset/train/images'
Note dataset download directory is '/workspace/Illegal-Parking-Detection/datasets'. You can update this in '/home/gitpod/.config/Ultralytics/settings.json'

# 4. 모델 추론

In [ ]:
model = YOLO('runs/detect/train/weights/best.pt')

In [ ]:
# 단일 이미지 추론
val_img_path = os.path.join(dataset_base_path, 'val/images/33_20210704_7454-0-0600.jpg')
results = model.predict(val_img_path, save=True, conf=0.25, device=0)

In [ ]:
# 전체 평가
metrics = model.val(
    data=os.path.join(root_path, 'data.yaml'),
    batch=16,
    imgsz=640,
    device=0
)
print(metrics)
# map50, map50-95, precision, recall 등 주요 지표 출력


# 1. YOLO는 차량을 감지하지만 그 차량이 불법인지는 안 알려줌
# 따라서 촬영 시간 / 위치 / 규칙을 바탕으로 후처리 판단이 꼭 필요
# 해당 함수는 이러한 판단 로직을 담당

In [ ]:
# 1. 불법 여부 판단 함수 추가
def is_illegal_parking(location_code, timestamp_str):
    rules = {
        "600": ("08:00", "20:00"),
        "700": ("07:00", "22:00"),
    }

    start_time, end_time = rules.get(location_code, ("00:00", "00:00"))
    timestamp = datetime.strptime(timestamp_str, "%Y%m%d_%H%M")

    start_dt = datetime.strptime(timestamp_str[:8] + "_" + start_time, "%Y%m%d_%H:%M")
    end_dt = datetime.strptime(timestamp_str[:8] + "_" + end_time, "%Y%m%d_%H:%M")

    return start_dt <= timestamp <= end_dt

In [ ]:
# 2. YOLO 모델 불러오기 및 단일 이미지 추론
model = YOLO('runs/detect/train/weights/best.pt')
val_img_path = os.path.join(dataset_base_path, 'val/images/33_20210704_7454-0-0600.jpg')
results = model.predict(val_img_path, save=True, conf=0.25, device=0)

In [ ]:
# 3. 파일이름에서 시간 추출 후 판단
filename = os.path.basename(val_img_path)  # 예: 33_20210704_7454-0-0600.jpg
timestamp_part = filename.split('_')[1] + "_" + filename.split('_')[-1].split('.')[0]  # "20210704_0600"
location_code = filename.split('_')[-1].split('-')[-1][:3]  # "600"

if is_illegal_parking(location_code, timestamp_part):
    print("불법주정차 감지!")
else:
    print("정상 주차")

# 불법 주정차 시간대 추가

In [ ]:
import os
from datetime import datetime
import json

# 1. 불법주정차 규칙 로딩 
def load_restriction_rules(json_path):
    with open(json_path, 'r', encoding='utf-8') as f:
        return json.load(f)

def is_illegal_parking(location_code, timestamp_str, rules_dict):
    rule = rules_dict.get(location_code)
    if not rule:
        return False

    start_time = rule.get("start_time", "00:00")
    end_time = rule.get("end_time", "00:00")

    timestamp = datetime.strptime(timestamp_str, "%Y%m%d_%H%M")
    start_dt = datetime.strptime(timestamp_str[:8] + "_" + start_time, "%Y%m%d_%H:%M")
    end_dt = datetime.strptime(timestamp_str[:8] + "_" + end_time, "%Y%m%d_%H:%M")

    return start_dt <= timestamp <= end_dt

#  2. 사용 
#  결과 이미지 경로
val_img_dir = "runs/detect/predict"
restriction_path = "./restricted_areas.json"
rules = load_restriction_rules(restriction_path)

# 결과 이미지 하나씩 순회하며 불법 여부 판단
for filename in os.listdir(val_img_dir):
    if filename.endswith(".jpg"):
        timestamp_part = filename.split('_')[1] + "_" + filename.split('_')[-1].split('.')[0]
        location_code = filename.split('_')[-1].split('-')[-1][:3]

        if is_illegal_parking(location_code, timestamp_part, rules):
            print(f" {filename}: 불법주정차 감지!")
        else:
            print(f" {filename}: 정상 주차")


# 불법 주정차 공휴일 및 주말 제외 추가

In [ ]:
import json
from datetime import datetime
import calendar

# 공휴일 리스트 (예시) - "YYYYMMDD" 형식
KOREAN_HOLIDAYS = [
    "20250721",  # 예시: 2025년 7월 21일
    "20250101",  # 신정
    "20250210",  # 설날
    "20250301",  # 삼일절
    
]

# 1. JSON에서 규칙 불러오기
def load_restriction_rules(json_path):
    with open(json_path, 'r', encoding='utf-8') as f:
        rules = json.load(f)
    return rules

# 2. 주말/공휴일 여부 확인
def is_weekend_or_holiday(timestamp):
    date_str = timestamp.strftime("%Y%m%d")
    weekday = timestamp.weekday()  # 월(0) ~ 일(6)

    return weekday >= 5 or date_str in KOREAN_HOLIDAYS

# 3. 특정 지역 + 시간에 대해 불법 여부 판단
def is_illegal_parking(location_code, timestamp_str, rules_dict):
    rule = rules_dict.get(location_code)
    if not rule:
        return False  # 등록되지 않은 지역은 불법 아님 처리

    start_time = rule.get("start_time", "00:00")
    end_time = rule.get("end_time", "00:00")

    timestamp = datetime.strptime(timestamp_str, "%Y%m%d_%H%M")
    
    #  주말/공휴일은 불법 아님 처리
    if is_weekend_or_holiday(timestamp):
        return False

    start_dt = datetime.strptime(timestamp_str[:8] + "_" + start_time, "%Y%m%d_%H:%M")
    end_dt = datetime.strptime(timestamp_str[:8] + "_" + end_time, "%Y%m%d_%H:%M")

    return start_dt <= timestamp <= end_dt


In [ ]:
# JSON 경로
restriction_path = "./restricted_areas.json"
rules = load_restriction_rules(restriction_path)

# 예시 파일명: 33_20250721_7454-0-0600.jpg
filename = os.path.basename(val_img_path)
timestamp_part = filename.split('_')[1] + "_" + filename.split('_')[-1].split('.')[0]  # "20250721_0600"
location_code = filename.split('_')[-1].split('-')[-1][:3]  # "600"

# 판단
if is_illegal_parking(location_code, timestamp_part, rules):
    print("불법주정차 감지!")
else:
    print("정상 주차 또는 예외일")


# 좌표를 도로명주소로 변환 / Kakao에서 Rest api 

In [ ]:
import requests
import json
from datetime import datetime

# 1. 좌표 → 주소 (도로명주소 + 행정코드) 변환
def reverse_geocode_kakao(x, y, api_key):
    url = "https://dapi.kakao.com/v2/local/geo/coord2address.json"
    headers = {"Authorization": f"KakaoAK {api_key}"}
    params = {"x": x, "y": y}
    
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        documents = response.json().get("documents")
        if documents:
            address_info = documents[0]["road_address"]
            road_name = address_info["address_name"]
            b_code = documents[0]["address"]["b_code"]  # 행정동 코드
            return road_name, b_code
    return None, None

# 2. 도로명주소 + 행정코드 → location_code 추출
def load_area_mapping(json_path="./area_mapping.json"):
    with open(json_path, "r", encoding="utf-8") as f:
        return json.load(f)

AREA_MAPPING = load_area_mapping()

def get_location_code(road_name, admin_code):
    info = AREA_MAPPING.get(road_name)
    if info and info.get("admin_code") == admin_code:
        return info.get("location_code")
    return None

# 3. location_code → 시간 기반 불법 여부 판단
def load_restricted_areas(json_path="./restricted_areas.json"):
    with open(json_path, "r", encoding="utf-8") as f:
        return json.load(f)

RESTRICTED_AREAS = load_restricted_areas()

def is_illegal_parking(location_code, current_time):
    location_info = RESTRICTED_AREAS.get(location_code)
    if not location_info:
        return False  # 제한 구역 아님

    day_name = current_time.strftime('%A').lower()
    now_time = current_time.time()
    time_range = location_info["restricted_times"].get(day_name)

    if not time_range:
        return False

    start_time = datetime.strptime(time_range["start"], "%H:%M").time()
    end_time = datetime.strptime(time_range["end"], "%H:%M").time()
    return start_time <= now_time <= end_time

# 4. 전체 흐름: YOLO 좌표 → 불법 여부 판단
def process_detection(x, y, api_key):
    road_name, admin_code = reverse_geocode_kakao(x, y, api_key)
    if not road_name or not admin_code:
        return False, "주소 변환 실패"

    location_code = get_location_code(road_name, admin_code)
    if not location_code:
        return False, "location_code 매핑 실패"

    now = datetime.now()
    if is_illegal_parking(location_code, now):
        return True, f"불법 주정차 (위치: {road_name})"
    else:
        return False, f"합법 주정차 (위치: {road_name})"


In [ ]:
if __name__ == "__main__":
    API_KEY = "YOUR_KAKAO_REST_API_KEY"  # 실제 카카오 REST API 키로 대체
    x, y = 127.0276, 37.4979  # 예: 강남역 근처 좌표

    result, message = process_detection(x, y, API_KEY)
    print(message)
